# Extração

In [33]:
import requests
import pandas as pd
from datetime import datetime as dt

In [34]:
def cache_data():
    url = "https://services1.arcgis.com/OlP4dGNtIcnD3RYf/arcgis/rest/services/Qualidade_do_ar_dados_horarios_2011_2018/FeatureServer/2/query?where=1%3D1&outFields=*&outSR=4326&f=json"
    response = requests.get(url)
    data = response.json()
    return data

data = cache_data()    

In [35]:
# Parsing the datasource
attributes = []

for attribute in data['features']:
    attributes.append(attribute['attributes'])

df = pd.DataFrame(attributes)
# contar quantos elementos não são None em cada coluna
print(df.count())

objectid            32000
data                32000
codnum              32000
estação             32000
chuva               31834
pres                31389
rs                  31344
temp                31621
ur                  31624
dir_vento           30927
vel_vento           30692
so2                 27671
no2                     0
hcnm                    0
hct                     0
ch4                     0
co                  31211
no                      0
nox                     0
o3                  30196
pm10                31169
pm2_5                   0
lat                 32000
lon                 32000
x_utm_sirgas2000    32000
y_utm_sirgas2000    32000
dtype: int64


# Transformação

In [36]:
# Convert ArcGIS UNIX to datetime
df['data'] = df['data'].apply(lambda x: dt.fromtimestamp(x/1000))
df

,objectid,data,codnum,estação,chuva,pres,rs,temp,ur,dir_vento,...,co,no,nox,o3,pm10,pm2_5,lat,lon,x_utm_sirgas2000,y_utm_sirgas2000
0,103080,2011-01-01 01:30:00,3,CA,0.2,1009.33,1.57,21.81,84.50,250.67,...,0.17,None,None,17.89,10.00,None,-22.908344,-43.178152,686853.7075,7.465470e+06
1,163230,2011-01-01 01:30:00,1,AV,0.0,1009.52,0.70,24.09,79.79,157.00,...,0.15,None,None,3.68,34.64,None,-22.965004,-43.180482,686537.0128,7.459198e+06
2,326722,2011-01-01 01:30:00,7,SC,0.0,1009.60,2.87,26.66,90.73,180.00,...,0.03,None,None,8.47,8.00,None,-22.897771,-43.221745,682395.7505,7.466695e+06
3,387059,2011-01-01 01:30:00,8,SP,0.0,1007.45,6.40,20.91,89.49,176.83,...,0.13,None,None,5.08,44.05,None,-22.924915,-43.232657,681240.2399,7.463703e+06
4,103081,2011-01-01 02:30:00,3,CA,0.0,1009.00,1.40,21.79,84.73,121.17,...,0.21,None,None,12.44,11.00,None,-22.908344,-43.178152,686853.7075,7.465470e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31995,395070,2011-11-30 20:30:00,8,SP,0.0,1005.63,11.10,27.91,71.86,251.17,...,0.68,None,None,3.77,43.83,None,-22.924915,-43.232657,681240.2399,7.463703e+06
31996,111040,2011-11-30 21:30:00,3,CA,0.0,1006.72,1.20,26.61,64.90,192.67,...,1.05,None,None,1.82,101.00,None,-22.908344,-43.178152,686853.7075,7.465470e+06
31997,171242,2011-11-30 21:30:00,1,AV,0.0,1006.98,0.90,26.01,76.94,189.00,...,0.46,None,None,0.00,66.94,None,-22.965004,-43.180482,686537.0128,7.459198e+06
31998,334734,2011-11-30 21:30:00,7,SC,0.0,1006.85,3.52,32.91,71.45,269.50,...,0.34,None,None,0.11,51.00,None,-22.897771,-43.221745,682395.7505,7.466695e+06


In [37]:
df['estação'].unique()

array(['CA', 'AV', 'SC', 'SP'], dtype=object)

In [38]:
#Dimensão data
dfData = df[['data']]

anos = dfData['data'].apply(lambda x: x.year)
meses = dfData['data'].apply(lambda x: x.month)
dias = dfData['data'].apply(lambda x: x.day)

dfData['ano'] = anos
dfData['mes'] = meses
dfData['dia'] = dias


dia_do_ano = dfData['data'].apply(lambda x: x.timetuple().tm_yday)
dia_da_semana = dfData['data'].apply(lambda x: x.weekday())
semana_do_ano = dfData['data'].apply(lambda x: x.isocalendar()[1])
semana_mes = dfData['data'].apply(lambda x: x.day//7)
e_ultimo_dia_do_mes = dfData['data'].apply(lambda x: x.day == x.days_in_month)

dfData['dia_ano'] = dia_do_ano
dfData['dia_semana'] = dia_da_semana
dfData['semana_ano'] = semana_do_ano
dfData['semana_mes'] = semana_mes
dfData['flag_ultimo_dia_mes'] = e_ultimo_dia_do_mes


clone = dfData.drop(columns='data')
clone = clone.drop_duplicates()
clone = clone.reset_index(drop=True)

clone['id_data'] = range(1,len(clone)+1)

dfData = dfData.merge(clone, on=['dia', 'mes', 'ano', 'dia_ano', 'dia_semana', 'semana_ano', 'semana_mes', 'flag_ultimo_dia_mes'])
dfData


/tmp/ipykernel_90289/269383624.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfData['ano'] = anos
/tmp/ipykernel_90289/269383624.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfData['mes'] = meses
/tmp/ipykernel_90289/269383624.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#return

,data,ano,mes,dia,dia_ano,dia_semana,semana_ano,semana_mes,flag_ultimo_dia_mes,id_data
0,2011-01-01 01:30:00,2011,1,1,1,5,52,0,False,1
1,2011-01-01 01:30:00,2011,1,1,1,5,52,0,False,1
2,2011-01-01 01:30:00,2011,1,1,1,5,52,0,False,1
3,2011-01-01 01:30:00,2011,1,1,1,5,52,0,False,1
4,2011-01-01 02:30:00,2011,1,1,1,5,52,0,False,1
...,...,...,...,...,...,...,...,...,...,...
31995,2011-11-30 20:30:00,2011,11,30,334,2,48,4,True,334
31996,2011-11-30 21:30:00,2011,11,30,334,2,48,4,True,334
31997,2011-11-30 21:30:00,2011,11,30,334,2,48,4,True,334
31998,2011-11-30 21:30:00,2011,11,30,334,2,48,4,True,334


In [39]:
# Dimensão horario
dfHorario = df[['data']]

horas = dfHorario['data'].apply(lambda x: x.hour)
minute = dfHorario['data'].apply(lambda x: x.minute)

dfHorario['hora'] = horas
dfHorario['minuto'] = minute

dfHorario = dfHorario.drop_duplicates(keep='first')

outro_df = pd.DataFrame({'hora': horas, 'minuto': minute})
# remover duplicatas
outro_df = outro_df.drop_duplicates(keep='first')

# gerar index
outro_df['id_horario'] = range(1,len(outro_df)+1)

# join outro_df com o dfHorario baseado na 'data'
dfHorario = dfHorario.merge(outro_df, on=['hora', 'minuto'])

dfHorario


/tmp/ipykernel_90289/827766409.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfHorario['hora'] = horas
/tmp/ipykernel_90289/827766409.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfHorario['minuto'] = minute


,data,hora,minuto,id_horario
0,2011-01-01 01:30:00,1,30,1
1,2011-01-02 01:30:00,1,30,1
2,2011-01-03 01:30:00,1,30,1
3,2011-01-04 01:30:00,1,30,1
4,2011-01-05 01:30:00,1,30,1
...,...,...,...,...
8007,2011-11-26 00:30:00,0,30,24
8008,2011-11-27 00:30:00,0,30,24
8009,2011-11-28 00:30:00,0,30,24
8010,2011-11-29 00:30:00,0,30,24


In [40]:
# dimensão estação

# ler csv estacao.csv
estacao = pd.read_csv('estacao.csv')
estacao['id_estacao'] = range(1,len(estacao)+1)

estacao

,sigla_estacao,zona_cidade,bairro,codnum_estacao,id_estacao
0,CA,Zona Central,Centro,3,1
1,AV,Zona Sul,Copacabana,1,2
2,SC,Zona Central,São Cristóvão,7,3
3,SP,Zona Norte,Tijuca,8,4
4,IR,Zona Norte,Irajá,5,5
5,BG,Zona Oeste,Bangu,2,6
6,CG,Zona Oeste,Campo Grande,4,7
7,PG,Zona Oeste,Pedra de Guaratiba,6,8


In [41]:
# dimensão parametro
df_parametros_enriquecidos = pd.read_csv('parametros_enriquecidos.csv')
lista_pedro = df.drop(columns=['objectid', 'codnum', 'estação', 'data']).columns
df_parametros_enriquecidos['nome_parametro'] = lista_pedro
df_parametros_enriquecidos['id_parametro'] = range(1,len(df_parametros_enriquecidos)+1)
df_parametros_enriquecidos = df_parametros_enriquecidos.drop(columns=['nome_parametro_2'])

df_parametros_enriquecidos


,unidade_medida_parametro,tipo_parametro,descritor_parametro,nome_parametro,id_parametro
0,mm,absoluto,"precipitação pluviométrica, em milímetros",chuva,1
1,mbar,absoluto,"pressão atmosférica, em milibares",pres,2
2,w/m2,absoluto,"radiação solar, em watts por metro quadrado",rs,3
3,°C,absoluto,"temperatura, em graus Celsius",temp,4
4,%,percentual,"umidade relativa do ar, em graus percentuais",ur,5
5,°,absoluto,"direção do vento, em graus, a partir do norte ...",dir_vento,6
6,m/s,absoluto,"velocidade do vento, em metros por segundo",vel_vento,7
7,μg/m^3,absoluto,"concentração de SO2 (dióxido de enxofre), em m...",so2,8
8,μg/m^3,absoluto,"concentração de NO2 (dióxido de nitrogênio), e...",no2,9
9,μg/m^3,absoluto,"oncentração de hidrocarbonetos não-metano, em ...",hcnm,10


In [42]:
dados_transformados = []

for idx, row in df.iterrows():
    for coluna in lista_pedro:
        dados_transformados.append({'nome_parametro': coluna, 'valor': row[coluna], 'data': row['data'], 'sigla_estacao': row['estação']})

df_transformado = pd.DataFrame(dados_transformados).dropna()

In [43]:
df_merged = df_transformado.merge(df_parametros_enriquecidos[['nome_parametro', 'id_parametro']], on='nome_parametro')
df_final = df_merged.drop('nome_parametro', axis=1).rename(columns={'nome_parametro': 'id_parametro'})

df_merged2 = df_final.merge(estacao[['sigla_estacao', 'id_estacao']], on='sigla_estacao')
df_final2 = df_merged2.drop('sigla_estacao', axis=1).rename(columns={'sigla_estacao': 'id_estacao'})

df_fatos_data_merged = df_final2.merge(dfData[['data', 'id_data']], on='data')
df_final3 = df_fatos_data_merged.merge(dfHorario[['data', 'id_horario']], on='data')
df_final3.drop(columns=['data'], inplace=True)

df_final3 = df_final3.drop_duplicates(keep='first')
df_final3

,valor,id_parametro,id_estacao,id_data,id_horario
0,2.000000e-01,1,1,1,1
4,1.009330e+03,2,1,1,1
8,1.570000e+00,3,1,1,1
12,2.181000e+01,4,1,1,1
16,8.450000e+01,5,1,1,1
...,...,...,...,...,...
1868856,-2.292492e+01,19,4,95,17
1868859,-4.323266e+01,20,4,95,17
1868862,6.812402e+05,21,4,95,17
1868865,7.463703e+06,22,4,95,17


In [44]:
# cleaning dimensions

dfData.drop(columns=['data'], inplace=True)
dfHorario.drop(columns=['data'], inplace=True)


In [52]:
dfHorario = dfHorario.drop_duplicates(keep='first')
dfHorario

dfFato = df_final3


In [46]:
dfData = dfData.drop_duplicates(keep='first')
dfData

,ano,mes,dia,dia_ano,dia_semana,semana_ano,semana_mes,flag_ultimo_dia_mes,id_data
0,2011,1,1,1,5,52,0,False,1
92,2011,1,2,2,6,52,0,False,2
188,2011,1,3,3,0,1,0,False,3
284,2011,1,4,4,1,1,0,False,4
380,2011,1,5,5,2,1,0,False,5
...,...,...,...,...,...,...,...,...,...
31528,2011,11,26,330,5,47,3,False,330
31624,2011,11,27,331,6,47,3,False,331
31720,2011,11,28,332,0,48,4,False,332
31816,2011,11,29,333,1,48,4,False,333


# Load

In [47]:
dfData.to_csv('dim_data.csv', index=False)
dfHorario.to_csv('dim_horario.csv', index=False)
estacao.to_csv('dim_estacao.csv', index=False)
df_parametros_enriquecidos.to_csv('dim_parametro.csv', index=False)
dfFato.to_csv('fato_qualidade_ar.csv', index=False)

In [50]:
omni = dfFato.merge(dfData, on='id_data')
omni = omni.merge(dfHorario, on='id_horario')
omni = omni.merge(estacao, on='id_estacao')
omni = omni.merge(df_parametros_enriquecidos, on='id_parametro')

omni.drop(columns=['id_data', 'id_horario', 'id_estacao', 'id_parametro'], inplace=True)

omni.to_csv('omni.csv', index=False)